In [ ]:
import mitsuba as mi
import numpy as np
from pathlib import Path

mi.set_variant("scalar_spectral")

thread = mi.Thread.thread()
fres = thread.file_resolver()
scene_path = str(Path("../resources/data/scenes/matpreview").resolve())
if scene_path not in fres:
    fres.append(scene_path)
fres

In [ ]:
def scene_matpreview(
    bsdf_matpreview=None, width=683, height=512, max_depth=-1, sampler="independent"
):
    if bsdf_matpreview is None:
        bsdf_matpreview = {
            "type": "plastic",
            "diffuse_reflectance": {
                "type": "rgb",
                "value": (0.940, 0.271, 0.361),
            },
            "int_ior": 1.9,
        }

    scene_dict = {
        "type": "scene",
        "integrator": {
            "type": "path",
            "max_depth": max_depth,
        },
        "camera": {
            "type": "perspective",
            "id": "camera",
            "fov_axis": "smaller",
            "focus_distance": 6.0,
            "fov": 28.8415,
            "to_world": mi.ScalarTransform4f.look_at(
                target=(3.04072, -2.85176, 2.80939),
                origin=(3.69558, -3.46243, 3.25463),
                up=(-0.317366, 0.312466, 0.895346),
            ),
            "sampler": {
                "type": sampler,
                "sample_count": 4,
            },
            "film": {
                "type": "hdrfilm",
                "id": "film",
                "width": width,
                "height": height,
                "pixel_format": "rgb",
                "rfilter": {"type": "gaussian"},
            },
        },
        "emitter": {
            "type": "envmap",
            "id": "emitter-envmap",
            "filename": "envmap.exr",
            "to_world": mi.ScalarTransform4f.rotate(axis=[0, 1, 0], angle=-180)
            @ mi.ScalarTransform4f(
                [
                    [-0.224951, -0.000001, -0.974370, 0.000000],
                    [-0.974370, 0.000000, 0.224951, 0.000000],
                    [0.000000, 1.000000, -0.000001, 8.870000],
                    [0.000000, 0.000000, 0.000000, 1.000000],
                ]
            ),
            "scale": 4.0,
        },
        "bsdf-diffuse": {"type": "diffuse", "reflectance": 0.18},
        "texture-checkerboard": {
            "type": "checkerboard",
            "color0": 0.4,
            "color1": 0.2,
            "to_uv": mi.ScalarTransform3f.scale(8.0),
        },
        "bsdf-plane": {
            "type": "diffuse",
            "reflectance": {"type": "ref", "id": "texture-checkerboard"},
        },
        "bsdf-matpreview": bsdf_matpreview,
        "shape-plane": {
            "type": "serialized",
            "filename": "matpreview.serialized",
            "shape_index": 0,
            "to_world": mi.ScalarTransform4f.rotate(axis=[0, 0, 1], angle=-4.3)
            @ mi.ScalarTransform4f(
                [
                    [3.38818, -4.06354, 0, -1.74958],
                    [4.06354, 3.38818, 0, 1.43683],
                    [0, 0, 5.29076, -0.0120714],
                    [0, 0, 0, 1],
                ]
            ),
            "bsdf": {"type": "ref", "id": "bsdf-plane"},
        },
        "shape-matpreview-interior": {
            "type": "serialized",
            "filename": "matpreview.serialized",
            "shape_index": 1,
            "to_world": mi.ScalarTransform4f(
                [
                    [1, 0, 0, 0],
                    [0, 1, 0, 0],
                    [0, 0, 1, 0.0252155],
                    [0, 0, 0, 1],
                ]
            ),
            "bsdf": {"type": "ref", "id": "bsdf-diffuse"},
        },
        "shape-matpreview-exterior": {
            "type": "serialized",
            "filename": "matpreview.serialized",
            "shape_index": 2,
            "to_world": mi.ScalarTransform4f(
                np.array(
                    [
                        [0.614046, 0.614047, 0, -1.78814e-07],
                        [-0.614047, 0.614046, 0, 2.08616e-07],
                        [0, 0, 0.868393, 1.02569],
                        [0, 0, 0, 1],
                    ]
                )
            )
            @ mi.ScalarTransform4f.translate([0, 0, 0.01]),
            "bsdf": {"type": "ref", "id": "bsdf-matpreview"},
        },
        "shape-sphere": {
            "type": "sphere",
            "center": [0, 0, 1.05],
            "radius": 0.94,
            "bsdf": {"type": "ref", "id": "bsdf-matpreview"},
        }
    }
    return scene_dict

In [ ]:
scene_dict = scene_matpreview(
    {"type": "measured", "filename": "themis-p2000-3-7035_spec.bsdf"},
    sampler="independent",
)
scene = mi.load_dict(scene_dict)
bmp = mi.Bitmap(mi.render(scene=scene, spp=16))
bmp